In [1]:
import pandas, copy

from tqdm import tqdm

import statsmodels.api as sm

from src.utils import build_exact_table, build_bootstrapped_table, create_predictions_table, plot_truthtables, plot_growth_boxplot

%load_ext autoreload
%autoreload 2

/Users/fowler/Library/Python/3.12/lib/python/site-packages/statsmodels/tools/tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  import scipy.linalg


In [2]:
who_drugs = list(pandas.read_csv('dat/drugs/who2_drugs.csv').drug)
plate_drugs = list(pandas.read_csv('dat/drugs/plate_drugs.csv').drug)
other_drugs = list(pandas.read_csv('dat/drugs/other_drugs.csv').drug)
mgit_drugs = list(pandas.read_csv('dat/drugs/mgit_drugs.csv').drug)

drug_names_table = pandas.read_csv("dat/drugs/drug_names_lookup.csv")
drug_names_table.set_index("DRUG", inplace=True)
drug_names_lookup = {}
for idx, row in drug_names_table.iterrows():
    drug_names_lookup[idx] = row.DRUG_NAME.capitalize()

In [3]:
PHENOTYPES = pandas.read_csv('dat/PHENOTYPES.csv')
PHENOTYPES.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
PHENOTYPES.sort_index(inplace=True)
PHENOTYPES[:2]

UNIQUEID  \
ENA_RUN_ACCESSION DRUG                                                 
ERR13286038       BDQ   site.10.subj.LA00559418.lab.LA00559418.iso.1   
                  LZD   site.10.subj.LA00559418.lab.LA00559418.iso.1   

                       BINARY_PHENOTYPE PHENOTYPE_QUALITY PHENOTYPE_METHOD  
ENA_RUN_ACCESSION DRUG                                                      
ERR13286038       BDQ                 R               NaN             MGIT  
                  LZD                 S               NaN             MGIT

In [4]:
RAW_PREDICTIONS = pandas.read_csv('dat/RAW_PREDICTIONS.csv')
RAW_PREDICTIONS.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
RAW_PREDICTIONS[:3]

PREDICTION
ENA_RUN_ACCESSION DRUG           
ERR4799790        AMI           R
                  BDQ           S
                  CAP           R

In [5]:
RAW_EFFECTS = pandas.read_csv('dat/RAW_EFFECTS.csv')
RAW_EFFECTS.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
RAW_EFFECTS.sort_index(inplace=True)
print(f"There are {RAW_EFFECTS.IS_NULL.sum()} null calls and {RAW_EFFECTS.IS_MINOR_ALLELE.sum()} minor allele calls in the main effects table")
RAW_EFFECTS[:3]

There are 503 null calls and 3118 minor allele calls in the main effects table


GENE      MUTATION PREDICTION  EPISTASIS  \
ENA_RUN_ACCESSION DRUG                                               
ERR13286038       BDQ   Rv0678  138_ins_g:38          R      False   
                  BDQ   Rv0678  141_ins_c:60          R      False   
                  CAP     tlyA          L11L          S      False   

                        IS_MINOR_ALLELE  IS_NULL  
ENA_RUN_ACCESSION DRUG                            
ERR13286038       BDQ              True    False  
                  BDQ              True    False  
                  CAP             False    False

Most of the null calls are in the *rrs* gene so will affect the aminoglycosides

In [6]:
RAW_EFFECTS[RAW_EFFECTS.IS_NULL].GENE.value_counts()

GENE
rrs       424
rpoB       20
katG       17
pncA       10
embB        9
Rv0678      8
eis         4
gid         2
fabG1       2
gyrB        2
tlyA        2
rplC        2
ethA        1
Name: count, dtype: int64

Whilst there are over 50 minor allele calls in genes that could affect resistance prediction for the fluorouinolones, rifampicin, bedaquline, pyrazinamide and ethambutol.

In [7]:
RAW_EFFECTS[RAW_EFFECTS.IS_MINOR_ALLELE].GENE.value_counts()[:15]

GENE
Rv0678    666
embB      501
rrs       368
gyrA      368
rpoB      169
gyrB      132
Rv2983    121
ethA      106
gid       104
fabG1      90
pncA       89
katG       88
rrl        53
eis        52
fgd1       43
Name: count, dtype: int64

Let's now build three EFFECTS tables that progressively include these nuances and from these build three PREDICTIONS tables that we can analyse by joining to PHENOTYPES

In [8]:
EFFECTS={}
EFFECTS[1] = copy.deepcopy(RAW_EFFECTS[~(RAW_EFFECTS.IS_MINOR_ALLELE) & ~(RAW_EFFECTS.IS_NULL)])
EFFECTS[1]['SET'] = 'basic'
EFFECTS[2] = copy.deepcopy(RAW_EFFECTS[~(RAW_EFFECTS.IS_MINOR_ALLELE)])
EFFECTS[2]['SET'] = 'nulls'
EFFECTS[3] = copy.deepcopy(RAW_EFFECTS)
EFFECTS[3]['SET'] = 'nulls+minors'
assert len(EFFECTS[3]) >  len(EFFECTS[2]) >  len(EFFECTS[1])

PREDICTIONS={}
for i in [1,2,3]:
    PREDICTIONS[i] = pandas.DataFrame(create_predictions_table(EFFECTS[i], who_drugs))

assert len(RAW_PREDICTIONS) == len(PREDICTIONS[1]) == len(PREDICTIONS[2]) == len(PREDICTIONS[3])

EFFECTS = pandas.concat(EFFECTS.values())
EFFECTS.reset_index(inplace=True)
EFFECTS.set_index(['SET','ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
EFFECTS.to_csv('dat/EFFECTS.csv')

PREDICTIONS= pandas.concat(PREDICTIONS.values())
PREDICTIONS.to_csv('dat/PREDICTIONS.csv')

100%|██████████| 55463/55463 [00:01<00:00, 34359.78it/s]


These tables each contain three scenarios (i) `basic` which ignores all null and minor alleles and is therefore closest to the WHOv2 catalogue as written, (ii) `nulls` which makes an `F` call at genetic loci associated with resistance where there are insufficient reads (two or fewer) and (iii) `nulls+minors` which adds in calls where there are three or more reads supporting a resistance-associated variant.

In [9]:
EFFECTS[:2]

GENE MUTATION PREDICTION  EPISTASIS  \
SET   ENA_RUN_ACCESSION DRUG                                        
basic ERR13286038       CAP   tlyA     L11L          S      False   
                        DLM   fgd1    K270M          U      False   

                              IS_MINOR_ALLELE  IS_NULL  
SET   ENA_RUN_ACCESSION DRUG                            
basic ERR13286038       CAP             False    False  
                        DLM             False    False

In [10]:
PREDICTIONS[:2]

PREDICTION
SET   ENA_RUN_ACCESSION DRUG           
basic ERR13286038       INH           R
                        RIF           R

Let's check that has all worked, first by picking a sample and looking at its rows in an EFFECTS table

In [11]:
check_sample = 'ERR13286038'
effects = copy.deepcopy(EFFECTS)
effects.reset_index(inplace=True)
effects = effects[(effects.SET=='nulls') & (effects.ENA_RUN_ACCESSION==check_sample)]
effects.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
effects

SET   GENE MUTATION PREDICTION  EPISTASIS  \
ENA_RUN_ACCESSION DRUG                                                
ERR13286038       CAP   nulls   tlyA     L11L          S      False   
                  DLM   nulls   fgd1    K270M          U      False   
                  EMB   nulls   embB    A409P          U      False   
                  EMB   nulls   embB    E405D          U      False   
                  ETH   nulls   ethA    Y211S          U      False   
                  ETH   nulls  fabG1    c-15t          R      False   
                  INH   nulls  fabG1    c-15t          R      False   
                  INH   nulls   katG    S315T          R      False   
                  LEV   nulls   gyrA     A90V          R      False   
                  LEV   nulls   gyrA     D94G          R      False   
                  LEV   nulls   gyrA     E21Q          S      False   
                  LEV   nulls   gyrA    G668D          S      False   
                  LEV   nulls   gyrA     S95T          S      False   
                  MXF   nulls   gyrA     A90V          R      False   
                  MXF   nulls   gyrA     D94G          R      False   
                  MXF   nulls   gyrA     E21Q          S      False   
                  MXF   nulls   gyrA    G668D          S      False   
                  MXF   nulls   gyrA     S95T          S      False   
                  PZA   nulls   pncA     H71Y          R      False   
                  RIF   nulls   rpoB    D103D          S      False   
                  RIF   nulls   rpoB    H445Y          R      False   
                  STM   nulls    gid    A134E          R      False   

                        IS_MINOR_ALLELE  IS_NULL  
ENA_RUN_ACCESSION DRUG                            
ERR13286038       CAP             False    False  
                  DLM             False    False  
                  EMB             False    False  
                  EMB             False    False  
                  ETH             False    False  
                  ETH             False    False  
                  INH             False    False  
                  INH             False    False  
                  LEV             False    False  
                  LEV             False    False  
                  LEV             False    False  
                  LEV             False    False  
                  LEV             False    False  
                  MXF             False    False  
                  MXF             False    False  
                  MXF             False    False  
                  MXF             False    False  
                  MXF             False    False  
                  PZA             False    False  
                  RIF             False    False  
                  RIF             False    False  
                  STM             False    False

Now compare that to the calculated per drug predictions

In [12]:
create_predictions_table(effects, who_drugs)

100%|██████████| 22/22 [00:00<00:00, 31600.92it/s]


PREDICTION
SET   ENA_RUN_ACCESSION DRUG           
nulls ERR13286038       INH           R
                        RIF           R
                        PZA           R
                        EMB           U
                        BDQ           S
                        LZD           S
                        MXF           R
                        LEV           R
                        CFZ           S
                        DLM           U
                        AMI           S
                        STM           R
                        ETH           R
                        KAN           S
                        CAP           S

Since we have a `RAW_PREDICTIONS` table parsed directly from the individual sample `json` files we can compare `PREDICTIONS['nulls+minors']` to that and they should be identical.

A direct way of doing that is to make an antibiogram for each sample (which will automatically be in alphabetical order of drug), and then look for samples where they are different.

In [13]:
def make_antibiogram(row):
    antibiogram=''
    for i in who_drugs:
        antibiogram += row[('PREDICTION',i)]
    return antibiogram

df1 = RAW_PREDICTIONS.unstack()
df1['ANTIBIOGRAM'] = df1.apply(make_antibiogram, axis=1)

PREDICTIONS.reset_index(inplace=True)
df2 = PREDICTIONS[PREDICTIONS.SET=='nulls+minors']
PREDICTIONS.set_index(['SET','ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
df2.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
df2=df2[['PREDICTION']].unstack()
# df2=df2.droplevel(0, axis=1)
df2['ANTIBIOGRAM'] = df2.apply(make_antibiogram, axis=1)

mask = (df1.ANTIBIOGRAM != df2.ANTIBIOGRAM)
assert len(df1[mask]) == 0, 'some samples have different antibiograms!'

Having the (alphabetical) antibiogram is useful, so let's calculate it for all three scenarios

In [14]:
df = PREDICTIONS.unstack()
df['ANTIBIOGRAM'] = df.apply(make_antibiogram, axis=1)
df = df[[('ANTIBIOGRAM','')]]
df = df.droplevel(1, axis=1)
df.reset_index(inplace=True)
df1 = df[df.SET=='basic']
df2 = df[df.SET=='nulls']
df3 = df[df.SET=='nulls+minors']
df1.set_index('ENA_RUN_ACCESSION', inplace=True)
df2.set_index('ENA_RUN_ACCESSION', inplace=True)
df3.set_index('ENA_RUN_ACCESSION', inplace=True)
df1 = df1.rename(columns={'SET':"SET1",'ANTIBIOGRAM':'ANTIBIOGRAM1'})
df2 = df2.rename(columns={'SET':"SET2",'ANTIBIOGRAM':'ANTIBIOGRAM2'})
df3 = df3.rename(columns={'SET':"SET3",'ANTIBIOGRAM':'ANTIBIOGRAM3'})
ANTIBIOGRAMS = df1.join(df2).join(df3)
ANTIBIOGRAMS.to_csv('dat/ANTIBIOGRAMS.csv')
ANTIBIOGRAMS[:3]

,SET1,ANTIBIOGRAM1,SET2,ANTIBIOGRAM2,SET3,ANTIBIOGRAM3
ENA_RUN_ACCESSION,,,,,,
ERR13286038,basic,RRRUSSRRSUSRRSS,nulls,RRRUSSRRSUSRRSS,nulls+minors,RRRURSRRRUSRRSS
ERR13286039,basic,RSSSSSSSSSSSSSS,nulls,RSSSSSSSSSSSSSS,nulls+minors,RSSSRSSSRSSSSSS
ERR13286042,basic,RRRRSURRSSRRRRR,nulls,RRRRSURRSSRRRRR,nulls+minors,RRRRSURRSSRRRRR


We can now look to see how many samples out of 2,663 change between scenarios

In [15]:
print(f"Calling nulls at resistance loci changes the antibiogram of {(ANTIBIOGRAMS.ANTIBIOGRAM1!=ANTIBIOGRAMS.ANTIBIOGRAM2).sum()} samples,\
whilst also calling minor alleles leads to {(ANTIBIOGRAMS.ANTIBIOGRAM2!=ANTIBIOGRAMS.ANTIBIOGRAM3).sum()} samples changing antibiogram")

Calling nulls at resistance loci changes the antibiogram of 44 samples,whilst also calling minor alleles leads to 375 samples changing antibiogram


In [16]:
ANTIBIOGRAMS[ANTIBIOGRAMS.ANTIBIOGRAM1!=ANTIBIOGRAMS.ANTIBIOGRAM2]

,SET1,ANTIBIOGRAM1,SET2,ANTIBIOGRAM2,SET3,ANTIBIOGRAM3
ENA_RUN_ACCESSION,,,,,,
ERR13288807,basic,RRSSSSSSSSSSSSS,nulls,RRSSSSSSSSSFSSS,nulls+minors,RRSSSSSSSSSFSSS
ERR2510689,basic,RRRRSSSSSSSRRSS,nulls,RRRRSSSSSSFRRFF,nulls+minors,RRRRSSSSSSFRRFF
ERR2515400,basic,RRRRSUSSSSSSRSS,nulls,RRRRSUSSSSSFRSS,nulls+minors,RRRRSUSSSRSFRSS
ERR4797493,basic,SSSSSSSSSUSSUSS,nulls,SFSSSSSSSUFSUFF,nulls+minors,SFSSSSSSSUFSUFF
ERR4797495,basic,SSSSSSSSSUSSSSS,nulls,SSSSSSSSSUSFSSS,nulls+minors,SSSSSSSSSUSFSSS
ERR4797833,basic,SSSSSSSSSSSSSSS,nulls,SSSSSSSSSSSFSSS,nulls+minors,SSSSSSSSSSSFSSS
ERR4798658,basic,RRRRSSRRSSRURRR,nulls,RRRRSSRRSSRFRRR,nulls+minors,RRRRSSRRSSRFRRR
ERR4798823,basic,SSSSSSSSSSSUSSS,nulls,FFFFFSSSSSFFFFF,nulls+minors,FFRRFSSSSSFFFFF
ERR4811314,basic,SSSSSSUUSSSSUSS,nulls,FSSSSSUUSSSSUSS,nulls+minors,FRSSSSUUSSSSUSS


In [17]:
results_drug_order = pandas.read_csv('dat/drugs/results_drugs.csv')
results_drug_order.set_index(['pDST method', 'drug'], inplace=True)
list(results_drug_order.index)

[('UKMYC', 'INH'),
 ('UKMYC', 'RIF'),
 ('MGIT', 'PZA'),
 ('UKMYC', 'EMB'),
 ('MGIT', 'BDQ'),
 ('UKMYC', 'LZD'),
 ('UKMYC', 'MXF'),
 ('UKMYC', 'LEV'),
 ('UKMYC', 'CFZ'),
 ('UKMYC', 'DLM'),
 ('UKMYC', 'AMI'),
 ('MGIT', 'STM'),
 ('UKMYC', 'ETH'),
 ('UKMYC', 'KAN'),
 ('MGIT', 'CAP')]

In [18]:
results={}
PREDICTIONS.reset_index(inplace=True)
PREDICTIONS.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)

for i in tqdm(['basic','nulls','nulls+minors', "tbprofiler"]):
    if i == 'tbprofiler':
        predictions = pandas.read_csv("dat/tbprofiler_PREDICTIONS.csv")
        predictions.set_index(['ENA_RUN_ACCESSION', 'DRUG'], inplace=True)
        df = copy.deepcopy(predictions.join(PHENOTYPES, how='inner'))
    else:
        predictions = PREDICTIONS[PREDICTIONS.SET==i]
        df = copy.deepcopy(predictions.join(PHENOTYPES, how='inner'))

    plate_df = df[df.PHENOTYPE_METHOD=='UKMYC']
    plate_df.reset_index(inplace=True)
    table = []
    table = build_exact_table(table, plate_df, plate_drugs, ['HIGH', 'ALL'], i, 'UKMYC', include_fails=False)
    table = build_bootstrapped_table(table,plate_df, plate_drugs, ['HIGH', 'ALL'], i, 'UKMYC', include_fails=False)

    mgit_df = df[df.PHENOTYPE_METHOD=='MGIT']
    mgit_df.reset_index(inplace=True)
    table = build_exact_table(table, mgit_df, mgit_drugs, ['ALL'], i, 'MGIT', include_fails=False)
    table = build_bootstrapped_table(table,mgit_df, mgit_drugs, ['ALL'], i, 'MGIT', include_fails=False)
    results[i] = pandas.DataFrame(table, columns=['set','drug','method','quality','dataset','sensitivity','sensitivity_sem','specificity','specificity_sem','PPV','PPV_sem', 'RR', 'SR', 'UR', 'RS', 'SS', 'US', 'Total'])

    results[i].set_index(['method','drug'],inplace=True)
    results[i] = results[i][results[i].index.isin(list(results_drug_order.index))]
    results[i].reset_index(inplace=True)
    results[i].drug = results[i].drug.astype('category')
    results[i].drug = results[i].drug.cat.set_categories(who_drugs)
    results[i].sort_values(['drug', 'dataset', 'quality'], inplace=True)
    results[i].set_index(['set','drug','method','dataset', 'quality'],inplace=True)

results = pandas.concat(results.values())
results.to_csv('dat/RESULTS.csv')
results

100%|██████████| 4/4 [00:33<00:00,  8.41s/it]


sensitivity  sensitivity_sem  \
set        drug method dataset        quality                                 
basic      INH  UKMYC  bootstrap-0    ALL        88.803089              NaN   
                                      HIGH       93.172691              NaN   
                       bootstrap-1    ALL        91.828794              NaN   
                                      HIGH       94.354839              NaN   
                       bootstrap-10   ALL        94.560669              NaN   
...                                                    ...              ...   
tbprofiler CAP  MGIT   bootstrap-7    ALL        76.250000              NaN   
                       bootstrap-8    ALL        70.068027              NaN   
                       bootstrap-9    ALL        67.816092              NaN   
                       bootstrapped50 ALL        73.778591         0.858111   
                       entire         ALL        74.308300         0.000000   

                                               specificity  specificity_sem  \
set        drug method dataset        quality                                 
basic      INH  UKMYC  bootstrap-0    ALL        95.020747              NaN   
                                      HIGH       96.414343              NaN   
                       bootstrap-1    ALL        94.650206              NaN   
                                      HIGH       96.825397              NaN   
                       bootstrap-10   ALL        95.785441              NaN   
...                                                    ...              ...   
tbprofiler CAP  MGIT   bootstrap-7    ALL        98.823529              NaN   
                       bootstrap-8    ALL        98.866856              NaN   
                       bootstrap-9    ALL        98.466258              NaN   
                       bootstrapped50 ALL        98.356579         0.165979   
                       entire         ALL        98.434783         0.000000   

                                                     PPV  PPV_sem     RR  \
set        drug method dataset        quality                              
basic      INH  UKMYC  bootstrap-0    ALL      95.041322      NaN    NaN   
                                      HIGH     96.265560      NaN    NaN   
                       bootstrap-1    ALL      94.779116      NaN    NaN   
                                      HIGH     96.694215      NaN    NaN   
                       bootstrap-10   ALL      95.358650      NaN    NaN   
...                                                  ...      ...    ...   
tbprofiler CAP  MGIT   bootstrap-7    ALL      96.825397      NaN    NaN   
                       bootstrap-8    ALL      96.261682      NaN    NaN   
                       bootstrap-9    ALL      95.934959      NaN    NaN   
                       bootstrapped50 ALL      95.263516  0.46716    NaN   
                       entire         ALL      95.431472  0.00000  188.0   

                                                 SR   UR   RS     SS   US  \
set        drug method dataset        quality                               
basic      INH  UKMYC  bootstrap-0    ALL       NaN  NaN  NaN    NaN  NaN   
                                      HIGH      NaN  NaN  NaN    NaN  NaN   
                       bootstrap-1    ALL       NaN  NaN  NaN    NaN  NaN   
                                      HIGH      NaN  NaN  NaN    NaN  NaN   
                       bootstrap-10   ALL       NaN  NaN  NaN    NaN  NaN   
...                                             ...  ...  ...    ...  ...   
tbprofiler CAP  MGIT   bootstrap-7    ALL       NaN  NaN  NaN    NaN  NaN   
                       bootstrap-8    ALL       NaN  NaN  NaN    NaN  NaN   
                       bootstrap-9    ALL       NaN  NaN  NaN    NaN  NaN   
                       bootstrapped50 ALL       NaN  NaN  NaN    NaN  NaN   
                       entire         ALL      62.0  3.0  9.0  566.0  0.0   



Now let's look at the number of Fail results when we allow these rules to contribute

In [19]:
EFFECTS.reset_index(inplace=True)
EFFECTS['GENE_MUTATION'] = EFFECTS.GENE + '_' + EFFECTS.MUTATION
print(f"There are {len(EFFECTS[(EFFECTS.SET=='nulls') & (EFFECTS.PREDICTION=='F')])} rows with a Fail call in the EFFECTS table (but these can be one variant affecting multiple drugs)")

df = EFFECTS[(EFFECTS.SET=='nulls') & (EFFECTS.PREDICTION=='F')]
df.set_index(['ENA_RUN_ACCESSION', 'GENE', 'MUTATION'], inplace=True)
print(f"Of the {len(df)} Fails, {df.index.duplicated().sum()} are duplicates")

df = df[~df.index.duplicated()]
df.reset_index(inplace=True)
print("And the most common genes are:")
df.GENE.value_counts()

There are 194 rows with a Fail call in the EFFECTS table (but these can be one variant affecting multiple drugs)
Of the 194 Fails, 17 are duplicates
And the most common genes are:


GENE
rrs       106
rpoB       20
katG       17
pncA       10
embB        9
Rv0678      4
gid         2
eis         2
tlyA        2
rplC        2
ethA        1
fabG1       1
gyrB        1
Name: count, dtype: int64

Looking at which the RAVs most likely to be affected we can see how STM is affected more than AMI and KAN 

In [20]:
df.GENE_MUTATION.value_counts()[:5]

GENE_MUTATION
rrs_c517x     48
rrs_a514x     46
embB_G406X     8
katG_S315X     7
rrs_g1484x     5
Name: count, dtype: int64

In [21]:
results.reset_index(inplace=True)
df = results[(results.set=="nulls+minors") & (results.method=="UKMYC") & (results.dataset=="entire") & (results.quality=='HIGH')]
df.sort_values(by='Total')

,set,drug,method,dataset,quality,sensitivity,sensitivity_sem,specificity,specificity_sem,PPV,PPV_sem,RR,SR,UR,RS,SS,US,Total
3535,nulls+minors,CFZ,UKMYC,entire,HIGH,12.820513,0.0,96.476510,0.0,32.258065,0.0,10.0,60.0,8.0,21.0,548.0,27.0,674.0
3223,nulls+minors,LZD,UKMYC,entire,HIGH,42.857143,0.0,99.847095,0.0,96.428571,0.0,27.0,33.0,3.0,1.0,599.0,54.0,717.0
3327,nulls+minors,MXF,UKMYC,entire,HIGH,90.128755,0.0,93.360161,0.0,86.419753,0.0,210.0,19.0,4.0,33.0,426.0,38.0,730.0
3639,nulls+minors,DLM,UKMYC,entire,HIGH,22.222222,0.0,100.000000,0.0,100.000000,0.0,14.0,33.0,16.0,0.0,523.0,161.0,747.0
3067,nulls+minors,EMB,UKMYC,entire,HIGH,89.867841,0.0,81.529851,0.0,67.326733,0.0,204.0,15.0,8.0,99.0,408.0,29.0,765.0
3431,nulls+minors,LEV,UKMYC,entire,HIGH,87.307692,0.0,96.584440,0.0,92.653061,0.0,227.0,26.0,7.0,18.0,466.0,43.0,788.0
3743,nulls+minors,AMI,UKMYC,entire,HIGH,84.426230,0.0,99.307958,0.0,98.095238,0.0,206.0,34.0,4.0,4.0,537.0,37.0,823.0
3899,nulls+minors,ETH,UKMYC,entire,HIGH,77.952756,0.0,85.478548,0.0,69.230769,0.0,198.0,20.0,36.0,88.0,397.0,121.0,861.0
2911,nulls+minors,RIF,UKMYC,entire,HIGH,96.583144,0.0,95.671982,0.0,95.711061,0.0,424.0,15.0,0.0,19.0,405.0,15.0,878.0
4003,nulls+minors,KAN,UKMYC,entire,HIGH,85.328185,0.0,98.546042,0.0,96.086957,0.0,221.0,32.0,6.0,9.0,564.0,46.0,879.0


In [22]:
conditions = [  (('basic', 'ALL'), ('basic', 'HIGH')),
                (('basic', 'ALL'), ('nulls+minors', 'ALL')),
                (('basic', 'ALL'), ('nulls+minors', 'HIGH')),
                (('basic', 'HIGH'), ('nulls+minors', 'HIGH')),
                (('nulls+minors', 'ALL'), ('nulls+minors', 'HIGH')),
            ]

df = results[results.dataset=='entire']

statistical_rows = []

for i in conditions:
    a = df[(df.set==i[0][0]) & (df.quality==i[0][1])]
    b = df[(df.set==i[1][0]) & (df.quality==i[1][1])]
    for drug in a.drug.unique():
        
        row1 = a[a.drug==drug]
        row2 = b[b.drug==drug]

        if not row1.empty and not row2.empty:
            tab = []
            tab.append([int(row1.RR), int(row1.SR+row1.UR), ],)
            tab.append([int(row2.RR), int(row2.SR+row2.UR), ],)        
            st = sm.stats.StratifiedTable([tab])
            st_results=st.summary().data
            for st_line in st_results:
                if st_line[0]=='Test of OR=1':
                    sens_pvalue=float(st_line[2])
            
            tab = []
            tab.append([int(row1.SS), int(row1.RS), ],)
            tab.append([int(row2.SS), int(row2.RS), ],)        
            st = sm.stats.StratifiedTable([tab])
            st_results=st.summary().data
            for st_line in st_results:
                if st_line[0]=='Test of OR=1':
                    spec_pvalue=float(st_line[2])

            tab = []
            tab.append([int(row1.RR), int(row1.RS), ],)
            tab.append([int(row2.RR), int(row2.RS), ],)        
            st = sm.stats.StratifiedTable([tab])
            st_results=st.summary().data
            for st_line in st_results:
                if st_line[0]=='Test of OR=1':
                    ppv_pvalue=float(st_line[2])


            tab = []
            tab.append([int(row1.SS), int(row1.SR+row1.UR), ],)
            tab.append([int(row2.SS), int(row2.SR+row2.UR), ],)        
            st = sm.stats.StratifiedTable([tab])
            st_results=st.summary().data
            for st_line in st_results:
                if st_line[0]=='Test of OR=1':
                    npv_pvalue=float(st_line[2])

            statistical_rows.append([i[0][0], i[0][1], i[1][0], i[1][1], drug, sens_pvalue, spec_pvalue, ppv_pvalue, npv_pvalue])
        else:
            statistical_rows.append([i[0][0], i[0][1], i[1][0], i[1][1], drug, None, None, None, None])

statistical_table = pandas.DataFrame(statistical_rows, columns=['dataset1', 'quality1', 'dataset2', 'quality2', 'drug', 'sensitivity', 'specificity', 'PPV', 'NPV'])
statistical_table.to_csv('dat/RESULTS_STATISTICAL_SIGNIFICANCE.csv')
statistical_table

/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_90466/1814392013.py:22: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tab.append([int(row1.RR), int(row1.SR+row1.UR), ],)
/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_90466/1814392013.py:23: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tab.append([int(row2.RR), int(row2.SR+row2.UR), ],)
/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_90466/1814392013.py:31: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tab.append([int(row1.SS), int(row1.RS), ],)
/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_90466/1814392013.py:32: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
0,basic,ALL,basic,HIGH,INH,0.118,0.522,0.430,0.174
1,basic,ALL,basic,HIGH,RIF,0.472,0.525,0.354,0.705
2,basic,ALL,basic,HIGH,PZA,NaN,NaN,NaN,NaN
3,basic,ALL,basic,HIGH,EMB,0.388,0.148,0.544,0.679
4,basic,ALL,basic,HIGH,BDQ,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
66,nulls+minors,ALL,nulls+minors,HIGH,AMI,0.005,0.783,0.957,0.027
67,nulls+minors,ALL,nulls+minors,HIGH,STM,NaN,NaN,NaN,NaN
68,nulls+minors,ALL,nulls+minors,HIGH,ETH,0.175,0.706,0.783,0.387
69,nulls+minors,ALL,nulls+minors,HIGH,KAN,0.014,0.765,0.993,0.051


In [23]:
statistical_table[statistical_table.drug=='KAN']

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
13,basic,ALL,basic,HIGH,KAN,0.029,0.793,0.993,0.098
28,basic,ALL,nulls+minors,ALL,KAN,0.334,0.613,0.689,0.441
43,basic,ALL,nulls+minors,HIGH,KAN,0.001,0.430,0.683,0.008
55,basic,HIGH,nulls+minors,HIGH,KAN,0.238,0.612,0.689,0.310
69,nulls+minors,ALL,nulls+minors,HIGH,KAN,0.014,0.765,0.993,0.051


In [24]:
statistical_table[(statistical_table.sensitivity<0.05) & (statistical_table.sensitivity>=0.01)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
10,basic,ALL,basic,HIGH,AMI,0.010,0.812,0.966,0.050
13,basic,ALL,basic,HIGH,KAN,0.029,0.793,0.993,0.098
61,nulls+minors,ALL,nulls+minors,HIGH,LZD,0.047,0.745,0.694,0.007
69,nulls+minors,ALL,nulls+minors,HIGH,KAN,0.014,0.765,0.993,0.051


In [25]:
statistical_table[(statistical_table.sensitivity<0.01) & (statistical_table.sensitivity>=0.001)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
23,basic,ALL,nulls+minors,ALL,CFZ,0.006,0.055,0.864,0.558
29,basic,ALL,nulls+minors,ALL,CAP,0.002,0.990,0.722,0.015
30,basic,ALL,nulls+minors,HIGH,INH,0.007,0.954,0.868,0.016
35,basic,ALL,nulls+minors,HIGH,LZD,0.004,0.833,0.979,0.002
36,basic,ALL,nulls+minors,HIGH,MXF,0.004,0.383,0.885,0.051
37,basic,ALL,nulls+minors,HIGH,LEV,0.003,0.759,0.863,0.016
43,basic,ALL,nulls+minors,HIGH,KAN,0.001,0.430,0.683,0.008
66,nulls+minors,ALL,nulls+minors,HIGH,AMI,0.005,0.783,0.957,0.027


In [26]:
statistical_table[(statistical_table.sensitivity<0.001) & (statistical_table.sensitivity>=0)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
19,basic,ALL,nulls+minors,ALL,BDQ,0.0,0.434,0.850,0.000
40,basic,ALL,nulls+minors,HIGH,AMI,0.0,0.522,0.718,0.004


In [27]:
statistical_table[(statistical_table.specificity<0.05)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV


In [28]:
statistical_table[(statistical_table.NPV<0.05) & (statistical_table.NPV>=0.01)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
5,basic,ALL,basic,HIGH,LZD,0.067,0.389,0.367,0.010
29,basic,ALL,nulls+minors,ALL,CAP,0.002,0.990,0.722,0.015
30,basic,ALL,nulls+minors,HIGH,INH,0.007,0.954,0.868,0.016
37,basic,ALL,nulls+minors,HIGH,LEV,0.003,0.759,0.863,0.016
66,nulls+minors,ALL,nulls+minors,HIGH,AMI,0.005,0.783,0.957,0.027


In [29]:
statistical_table[(statistical_table.NPV<0.01) & (statistical_table.NPV>=0.001)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
35,basic,ALL,nulls+minors,HIGH,LZD,0.004,0.833,0.979,0.002
40,basic,ALL,nulls+minors,HIGH,AMI,0.000,0.522,0.718,0.004
43,basic,ALL,nulls+minors,HIGH,KAN,0.001,0.430,0.683,0.008
61,nulls+minors,ALL,nulls+minors,HIGH,LZD,0.047,0.745,0.694,0.007


In [30]:
statistical_table[(statistical_table.NPV<0.001) & (statistical_table.NPV>=0)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
8,basic,ALL,basic,HIGH,CFZ,0.566,0.950,0.027,0.0
9,basic,ALL,basic,HIGH,DLM,0.116,0.372,0.395,0.0
19,basic,ALL,nulls+minors,ALL,BDQ,0.000,0.434,0.850,0.0
38,basic,ALL,nulls+minors,HIGH,CFZ,0.094,0.083,0.033,0.0
39,basic,ALL,nulls+minors,HIGH,DLM,0.065,0.372,0.378,0.0
64,nulls+minors,ALL,nulls+minors,HIGH,CFZ,0.808,0.902,0.010,0.0
65,nulls+minors,ALL,nulls+minors,HIGH,DLM,0.091,0.372,0.391,0.0


In [31]:
statistical_table[(statistical_table.PPV<0.05) & (statistical_table.PPV>=0.01)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
8,basic,ALL,basic,HIGH,CFZ,0.566,0.950,0.027,0.0
38,basic,ALL,nulls+minors,HIGH,CFZ,0.094,0.083,0.033,0.0
64,nulls+minors,ALL,nulls+minors,HIGH,CFZ,0.808,0.902,0.010,0.0


In [32]:
statistical_table[(statistical_table.PPV<0.01) & (statistical_table.PPV>=0.001)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV


In [33]:
statistical_table[(statistical_table.PPV<0.001) & (statistical_table.PPV>=0)]

,dataset1,quality1,dataset2,quality2,drug,sensitivity,specificity,PPV,NPV
